In [25]:
!pip install torch-pruning torchprofile ultralytics

In [8]:
# for quantization but not working
# !git clone https://github.com/NVIDIA-AI-IOT/torch2trt
#%cd torch2trt
#!python torch2trt/setup.py install

Cloning into 'torch2trt'...
remote: Enumerating objects: 4452, done.
remote: Counting objects: 100% (4452/4452), done.
remote: Compressing objects: 100% (1848/1848), done.
remote: Total 4452 (delta 2593), reused 4320 (delta 2547), pack-reused 0
Receiving objects: 100% (4452/4452), 7.75 MiB | 13.95 MiB/s, done.
Resolving deltas: 100% (2593/2593), done.
/content/torch2trt
python3: can't open file '/content/torch2trt/torch2trt/setup.py': [Errno 2] No such file or directory


In [26]:
import sys
import torch.nn.functional as F
import torch
import torch_pruning as tp
import torchprofile
import os
import copy
import random
import numpy as np
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import *
from tqdm.auto import tqdm
from functools import partial
import torchvision.models as models
import ultralytics
#from torch2trt import torch2trt
#assert torch.cuda.is_available()

In [61]:
# importing shared folder and existing code from HackIA24_input
from google.colab import drive
drive.mount('/content/drive')


sys.path.append( '/content/drive/MyDrive/HackIA_2024_GRP_2/Developpement/HackIA24_Input/compression_scripts' )
from pruning_modified import get_pruner, progressive_pruning_speedup, progressive_pruning_speedup, progressive_pruning_compression_ratio, train
sys.path.append( '/content/drive/MyDrive/HackIA_2024_GRP_2/Developpement/HackIA24_Input/Benchmark_scripts' )
from benchmark import benchmark, benchmark_yolo

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
bases_path = "/content/drive/MyDrive/HackIA_2024_GRP_2/Dataset/"#"/content/drive/MyDrive/HackIA_2024_GRP_2/Dataset/"
models_path = "/content/drive/MyDrive/HackIA_2024_GRP_2/Models/"
models_to_prune_path = "/content/drive/MyDrive/HackIA_2024_GRP_2/ModelsToPrune/"

train_dataset_path = bases_path + "personal_Cleansed"
test_dataset_path = bases_path + "test/"

#best_model = 'full_model_MobileNet_V3_Small_Weights_test_acc_0.923.pth'
#best_model = "ResNet150-model_2_2.pt"
#best_model = "ResNet150-model_2_98.pt"
#best_model = "model_vgg1670.87.pt"
best_model = "model-ResNet152-LastOne.pt"
#best_model = "Yolov8.pt"
#best_model = 'firenet.pth'
best_model_path =  os.path.join( models_to_prune_path, best_model )

In [62]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### CONFIG ###
model = torch.load( best_model_path )
# model loading without architecture > rebuilding architecture first
#weights = models.MobileNet_V3_Small_Weights.IMAGENET1K_V1
#base_model = models.mobilenet_v3_small(weights=weights)

#print( base_model )

#model.classifier[3] = nn.Linear(model.classifier[3].in_features, 3)

#model = torch.load( best_model_path ).to(device)
#state_dict = torch.load( best_model_path )#, map_location = torch.device( 'cpu') )
#model.load_state_dict(state_dict)


random_seed = 42

# pruning parameters
method = "group_sl" # choices "random", "l1", "lamp", "slim", "group_norm", "group_sl"
speed_up = 2 # speed up ratio based on MACs
compression_ratio = 4 # compression ratio based on model size
global_pruning = True # if True: global pruning else: local
iterative_steps = 400 # pruning steps
max_sparsity = 1 # max sparsity

num_classes = 3 # to avoid pruning last layer

# Fixer le seed pour la reproductibilité
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# training parameters for iterative pruning
batchSize	 = 32                                                             # define the batch size
trainSplit   = 0.8                                                            # define the ratio of data to use for training
testSplit    = 0.1                                                            # define the ratio of data to use for testing
validSplit   = 0.1                                                            # the rest is used for validation
epochs		 = 10                                                            # define the number of epoch
criterion    = nn.CrossEntropyLoss()                                          # define the loss function
learnRate	 = 0.01                                                           # define the learning rate
optimizer    = Adam(model.parameters(), lr=learnRate)                # define the optimizer
imgSize      = 224

In [63]:
# Preparing Torch datasets

# Preparing data transformations (composition of transforms)
transform   = transforms.Compose([
								transforms.RandomResizedCrop(imgSize),	# Resize the picture
								transforms.ToTensor()					# Transform the picture as a tensor
])
# Preparing target (label) transformations (composition of transforms)
target_transform = transforms.Compose([
		# Only a function that 1) transform an int (label) to a tensor, 2) encode the tensor (one-hot), 3) convert the tensor to float
		lambda label: torch.nn.functional.one_hot(torch.tensor( label ), num_classes=3).float()
])
# Create the dataset based on the path and apply the transformations defined above
dataset   = datasets.ImageFolder(
									train_dataset_path,
									transform=transform,
									target_transform=target_transform
)

# Compute the number of pictures that will be used for train, val and test datasets
trainlen	= int(trainSplit * len(dataset))
testlen		= int(testSplit  * len(dataset))
validlen	= len(dataset) - trainlen - testlen
# Then random split indices of the dataset into subsets: train, valid and test
# The results are sampler (dataset indices generator)
trainset,validset,testset = torch.utils.data.random_split(range(len(dataset)), [trainlen, validlen, testlen])

# create a loader foreach subset based on its sampler with a batch_size
trainloader = DataLoader(
						dataset,
						batch_size=batchSize,
						sampler=torch.utils.data.SubsetRandomSampler(trainset), # Use the SubsetRandomSampler to random indices at each epoch
)

valloader = DataLoader(
						dataset,
						batch_size=batchSize,
						sampler=validset
)

testloader = DataLoader(
						dataset,
						batch_size=batchSize,
						sampler=testset
)

In [64]:
print( model )

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [65]:
# Avant pruning
example_input = torch.rand(1, 3, 224, 224)
model = model.to( 'cuda:0' )
example_input = example_input.to( 'cuda:0')

start_macs, start_params = tp.utils.count_ops_and_params(model, example_input)

pruner = get_pruner(model, example_input, method, global_pruning, iterative_steps, max_sparsity, num_classes )
print(f'Pruning method = {method}')

# pruning using compression ratio as objective
progressive_pruning_compression_ratio(pruner, model, compression_ratio, example_input)

# pruning using speed up ratio as objective (Number of MACs)
print( model )
#progressive_pruning_speedup(pruner, model, speed_up, example_input)
pruned_macs, pruned_params = tp.utils.count_ops_and_params(model, example_input)

# Results
print('----- Results before fine tuning -----')
print(f'Params: {start_params/1e6:.2f} M => {pruned_params/1e6:.2f} M')
print(f'MACs: {start_macs/1e6:.2f} M => {pruned_macs/1e6:.2f} M')
print('')


/usr/local/lib/python3.10/dist-packages/torch_pruning/pruner/algorithms/metapruner.py:87: UserWarning: ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.
  warnings.warn("ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.")
/usr/local/lib/python3.10/dist-packages/torch_pruning/pruner/algorithms/metapruner.py:90: UserWarning: ch_sparsity_dict is deprecated in v1.3.0. Please use pruning_ratio_dict instead.
  warnings.warn("ch_sparsity_dict is deprecated in v1.3.0. Please use pruning_ratio_dict instead.")


Pruning method = group_sl


RuntimeError: The size of tensor a (1015) must match the size of tensor b (1024) at non-singleton dimension 1

In [53]:
splitted = best_model.split( '.' )
pruned_model_path = models_to_prune_path + best_model[ :-3 ] + "-pruned" + best_model[ -3: ]
print( pruned_model_path )

/content/drive/MyDrive/HackIA_2024_GRP_2/ModelsToPrune/model-ResNet152-LastOne-pruned.pt


In [37]:
torch.save( model, pruned_model_path )

In [49]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#pruned_model = torch.load( 'pruned_test.pt' )
train( model, trainloader, testloader, epochs, learnRate, save = pruned_model_path)

  0%|          | 0/43 [00:00<?, ?it/s]

eval:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/10 | Val acc: 41.52 | Val loss: 1.0925 | LR: 0.010000


  0%|          | 0/43 [00:00<?, ?it/s]

eval:   0%|          | 0/6 [00:00<?, ?it/s]

...

eval:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 9/10 | Val acc: 28.07 | Val loss: 1.1050 | LR: 0.010000


  0%|          | 0/43 [00:00<?, ?it/s]

eval:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 10/10 | Val acc: 28.07 | Val loss: 1.1008 | LR: 0.010000
Best val acc: 41.52


In [40]:
print( pruned_model_path )

/content/drive/MyDrive/HackIA_2024_GRP_2/Models/model_vgg1670.8-pruned7.pt


In [34]:
input_data = torch.rand(1, 3, 224, 224).to(device)

#weights = models.MobileNet_V3_Small_Weights.IMAGENET1K_V1
#start_model = models.mobilenet_v3_small(weights=weights)
#model.classifier[3] = nn.Linear(model.classifier[3].in_features, 3)

#model = torch.load( best_model_path ).to(device)
#state_dict = torch.load( best_model_path )#, map_location = torch.device( 'cpu') )
#start_model.load_state_dict(state_dict)

#state_dict = torch.load( pruned_model_path )#, map_location = torch.device( 'cpu') )
#model.load_state_dict( state_dict )
benchmark( model, input_data )

Inference time CPU (ms/image):314.418 ms +/- 132.188 ms
FPS CPU: 3.1804778539257352
Inference time GPU (ms/image): 5.378 ms +/- 0.225 ms
FPS GPU: 185.93201445973142
Nombre de paramètres: 94.849 M
Taille du modèle: 361.818 MiB
Nombre de MACs: 7598.517 M


(314.41816165004184,
 132.18775602692324,
 5.378309931755066,
 0.22487013996681013)